In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!ls '/content/gdrive/MyDrive/cll788_Term Paper'

In [ ]:
path = '/content/gdrive/MyDrive/cll788_Term Paper'

In [ ]:
import pickle
import os

In [ ]:
tm15_to_read = open(path + '/' 'timeseries_dict15.pickle', 'rb')
loaded_tm15 = pickle.load(tm15_to_read)
print(len(loaded_tm15))

In [ ]:
key15 = list(loaded_tm15.keys())

In [ ]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, Conv1d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn as nn
from torch.utils.data import DataLoader

In [ ]:
x = []
for key in key15:
  l = []
  for e in loaded_tm15[key]:
    l.append(e[0])
  l = np.array(l)
  x.append(l)
xn = np.array(x)
xn.shape

In [ ]:
train_x = torch.from_numpy(xn).to(device)
train_x.shape

In [ ]:
class RNNnet(nn.Module):

    def __init__(self, input_size, hidden_size):
        super(RNNnet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn = nn.GRU(input_size = input_size, hidden_size = hidden_size, batch_first = True)
  
    def rnn_implement(self, x):

        h_0 = torch.zeros((self.hidden_size, 1))
        output, h_n = self.rnn(x, h_0)
        output = output.mean(dim = 1)

    return output

In [ ]:
class CNNnet(nn.Module):

    def __init__(self, out_channels, kernel_size):
        super(CNNnet, self).__init__()

        self.cnn = nn.Conv1d(in_channels = 1, out_channels = out_channels, kernel_size = kernel_size)

    def cnn_implement(self, x):

        output = nn.functional.relu(self.cnn(x))

    return output

In [ ]:
class MLP(nn.Module):
  
    def __init__(self, input_size, hidden_size, layers, output_size):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(nn.Linear(input_size, hidden_size), ReLU(inplace = True),
                                 nn.Linear(hidden_size, hidden_size), ReLU(inplace = True),
                                 nn.Linear(hidden_size, hidden_size), ReLU(inplace = True),
                                 nn.Linear(hidden_size, output_size), Softmax(inplace = True))
    def forward(self, x):

        output = self.mlp(x)

    return output

In [ ]:
class EnsembleModel(nn.Module):

    def __init__(self, input_size, hidden_size, out_channels, kernel_size, mlp_hidden_size, mlp_layers, output_size):
        super(EnsembleModel, self).__init__()
        self.rnn = RNNnet(input_size = input_size, hidden_size = hidden_size)
        self.cnn = CNNnet(out_channels = out_channels, kernel_size = kernel_size)
        self.mlp = MLP(input_size = (mlp_hidden_size + out_channels), hidden_size = mlp_hidden_size, layers = mlp_layers, 
                       output_size = output_size)

    def forward(self, x):

        h_0 = torch.zeros((self.hidden_size, 1))
        output_rnn = self.rnn(x, h0)
        output_cnn = self.cnn(x)
        output_mlp = self.mpl(torch.cat([output_rnn, output_cnn], dim = 1))

    return output_mlp

In [ ]:
n = train_x.shape[1]
n

In [ ]:
batch_size = 32

In [ ]:
gru_hidden_size = 32

In [ ]:
h0 = torch.zeros(1, batch_size, gru_hidden_size, dtype=torch.float, requires_grad = False).to(device)

In [ ]:
model1 = RNNnet(train_x.shape[2], gru_hidden_size)
optimizer1 = Adam(params = model1.parameters())
criterion1 = CrossEntropyLoss(reduction = 'mean')
model1 = model1.to(device)

In [ ]:
model2 = CNNnet()
optimizer2 = Adam(params = model2.parameters())
criterio2 = CrossEntropyLoss(reduction = 'mean')

In [ ]:
model = EnsembleModel()
optimizer = Adam(params = model.parameters())
criterion = CrossEntropyLoss(reduction = 'mean')

In [ ]:
epochs = 200

In [ ]:
train_loader = DataLoader(train_x, batch_size = batch_size, shuffle = True, drop_last = True)

In [ ]:
for ep in tqdm(range(epochs), desc = "epochs"):
    for i, (inputs) in enumerate(train_loader):
        optimizer1.zero_grad()
        inputs = torch.tensor(inputs, dtype=torch.float, requires_grad = False).to(device)
        labels = torch.tensor(labels, dtype = torch.long, requires_grad = False).to(device)
        output1 = model1(inputs, h0)
        loss1 = criterion1(output1, labels)
        loss1.backward()
        optimizer1.step()